# LangGraph + Apify MCP 연동 튜토리얼

이 노트북에서는 LangGraph Agent가 Apify MCP 서버를 통해 Instagram Reel을 스크래핑하고 분석하는 방법을 학습합니다.

## 목표
**"인플루언서 ID만 입력하면 → Agent가 알아서 크롤링 → 결과 분석까지"**

```
User: "natgeo 인스타 릴스 최근 트렌드 분석해줘"
    ↓
LangGraph Agent
    ↓ MCP Protocol
Apify MCP Server (mcp.apify.com)
    ↓ Actor 실행
Instagram Reel Scraper
    ↓
결과 반환 → Agent가 분석/요약
```

## 📋 학습 내용
1. 방법 1: `langchain-apify` - ApifyActorsTool 직접 사용
2. 방법 2: `langchain-mcp-adapters` - MCP 서버 연결
3. 커스텀 LangGraph Agent 구축
4. 실전 예제: 인플루언서 분석 Agent

---

## 0️⃣ 환경 설정

In [ ]:
# 필요한 패키지 설치
!pip install -q langgraph langchain-apify langchain-openai langchain-mcp-adapters pandas

In [ ]:
import os

# API 키 설정
os.environ["APIFY_API_TOKEN"] = "<YOUR_APIFY_TOKEN>"  # https://console.apify.com/settings/integrations
os.environ["OPENAI_API_KEY"] = "<YOUR_OPENAI_KEY>"    # https://platform.openai.com/api-keys

print("✅ 환경 변수 설정 완료!")

---

## 1️⃣ 방법 1: langchain-apify (가장 간단)

`langchain-apify` 패키지의 `ApifyActorsTool`을 사용하면 Apify Actor를 LangChain Tool로 바로 변환할 수 있습니다.

`에러 : 버전 충돌로 사용 불가. 할려면 따로 해야함.`

### 1.1 기본 사용법

In [ ]:
from langchain_apify import ApifyActorsTool
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage

# LLM 초기화
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Apify Actor를 Tool로 변환
instagram_reel_tool = ApifyActorsTool("apify/instagram-reel-scraper")

print(f"📦 Tool 이름: {instagram_reel_tool.name}")
print(f"📝 Tool 설명: {instagram_reel_tool.description[:200]}...")

In [ ]:
# ReAct Agent 생성
tools = [instagram_reel_tool]
agent = create_react_agent(llm, tools)

print("✅ ReAct Agent 생성 완료!")

In [ ]:
# Agent 실행!
query = "natgeo 인스타그램 계정의 최근 릴스 5개를 가져와서 어떤 콘텐츠가 인기있는지 분석해줘"

print(f"🔍 Query: {query}")
print("=" * 60)
print("🤖 Agent 실행 중...\n")

for state in agent.stream(
    {"messages": [HumanMessage(content=query)]},
    stream_mode="values"
):
    state["messages"][-1].pretty_print()

### 1.2 여러 Actor 조합하기

웹 브라우저 + Instagram Scraper를 함께 사용하는 예제

In [ ]:
# 여러 Apify Actor를 Tool로 추가
browser_tool = ApifyActorsTool("apify/rag-web-browser")      # 웹 검색/브라우징
instagram_tool = ApifyActorsTool("apify/instagram-reel-scraper")  # Instagram Reel

# 멀티 툴 Agent
multi_tools = [browser_tool, instagram_tool]
multi_agent = create_react_agent(llm, multi_tools)

print("🛠️ 사용 가능한 도구들:")
for tool in multi_tools:
    print(f"  - {tool.name}")

In [ ]:
# 복합 작업 실행
complex_query = """
1. 먼저 웹에서 "natgeo instagram followers" 검색해서 팔로워 수 확인해줘
2. 그 다음 natgeo의 최근 릴스 3개를 분석해줘
3. 팔로워 대비 engagement rate를 추정해줘
"""

print(f"🔍 Complex Query:\n{complex_query}")
print("=" * 60)

# 실행 (주석 해제하여 실행)
# for state in multi_agent.stream(
#     {"messages": [HumanMessage(content=complex_query)]},
#     stream_mode="values"
# ):
#     state["messages"][-1].pretty_print()

---

## 2️⃣ 방법 2: langchain-mcp-adapters (MCP 프로토콜)

MCP(Model Context Protocol)를 통해 Apify 서버에 직접 연결하는 방법입니다.

### 2.1 MCP 서버 연결 방식 비교

| 방식 | URL | 용도 |
|------|-----|------|
| HTTP (Remote) | `https://mcp.apify.com` | 프로덕션, 간편 설정 |
| stdio (Local) | `npx @apify/actors-mcp-server` | 로컬 개발, 디버깅 |

In [4]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
import os

# MCP 클라이언트 설정
APIFY_TOKEN = os.environ["APIFY_API_TOKEN"]

# HTTP 방식으로 Apify MCP 서버 연결
mcp_config = {
    "apify": {
        "url": f"https://mcp.apify.com?tools=apify/instagram-reel-scraper",
        "transport": "http",
        "headers": {
            "Authorization": f"Bearer {APIFY_TOKEN}"
        }
    }
}

print("📋 MCP 서버 설정:")
print(f"  - URL: https://mcp.apify.com")
print(f"  - Tools: apify/instagram-reel-scraper")

📋 MCP 서버 설정:
  - URL: https://mcp.apify.com
  - Tools: apify/instagram-reel-scraper


In [5]:
async def run_mcp_agent(query: str):
    """
    MCP 서버에 연결하여 Agent 실행
    """
    async with MultiServerMCPClient(mcp_config) as client:
        # MCP 서버에서 도구 가져오기
        tools = await client.get_tools()
        print(f"🛠️ MCP에서 로드된 도구 수: {len(tools)}")
        
        for tool in tools:
            print(f"  - {tool.name}")
        
        # Agent 생성 및 실행
        # llm = ChatOpenAI(model="gpt-5-nano", temperature=0)
        llm = ChatGoogleGenerativeAI(model="gemini-3.0-flash-preview", temperature=0)
        agent = create_react_agent(llm, tools)
        
        result = await agent.ainvoke({"messages": [HumanMessage(content=query)]})
        return result

print("✅ MCP Agent 함수 정의 완료!")

✅ MCP Agent 함수 정의 완료!


In [ ]:
# MCP Agent 실행 (Jupyter에서는 await 직접 사용 가능)
result = await run_mcp_agent("ddobini619 인스타그램 릴스 5개 분석해줘")
print(result["messages"][-1].content)

NotImplementedError: As of langchain-mcp-adapters 0.1.0, MultiServerMCPClient cannot be used as a context manager (e.g., async with MultiServerMCPClient(...)). Instead, you can do one of the following:
1. client = MultiServerMCPClient(...)
   tools = await client.get_tools()
2. client = MultiServerMCPClient(...)
   async with client.session(server_name) as session:
       tools = await load_mcp_tools(session)

### 2.2 stdio 방식 (로컬 MCP 서버)

로컬에서 MCP 서버를 실행하여 연결하는 방식입니다.

In [7]:
# stdio 방식 설정 (Node.js 필요)
stdio_config = {
    "apify-local": {
        "command": "npx",
        "args": [
            "-y", 
            "@apify/actors-mcp-server",
            "--tools", "apify/instagram-reel-scraper,apify/rag-web-browser"
        ],
        "transport": "stdio",
        "env": {
            "APIFY_TOKEN": APIFY_TOKEN
        }
    }
}

print("📋 stdio MCP 설정 (로컬):")
print(f"  - Command: npx -y @apify/actors-mcp-server")
print(f"  - Tools: instagram-reel-scraper, rag-web-browser")

📋 stdio MCP 설정 (로컬):
  - Command: npx -y @apify/actors-mcp-server
  - Tools: instagram-reel-scraper, rag-web-browser


---

## 3️⃣ 커스텀 LangGraph Agent 구축

`create_react_agent` 대신 직접 StateGraph를 구축하여 더 세밀한 제어가 가능합니다.

In [8]:
from typing import Annotated, TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

# State 정의
class AgentState(TypedDict):
    messages: Annotated[list, add_messages]
    instagram_data: dict  # 스크래핑 결과 저장
    analysis_result: str  # 분석 결과

print("✅ AgentState 정의 완료!")

✅ AgentState 정의 완료!


In [9]:
# Tool 설정
from langchain_apify import ApifyActorsTool
from langchain_openai import ChatOpenAI

instagram_tool = ApifyActorsTool("apify/instagram-reel-scraper")
tools = [instagram_tool]

# LLM with Tools
# llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
llm = ChatGoogleGenerativeAI(model="gemini-3.0-flash-preview", temperature=0)
llm_with_tools = llm.bind_tools(tools)

# System Prompt
SYSTEM_PROMPT = """당신은 Instagram 인플루언서 분석 전문가입니다.

사용 가능한 도구:
- instagram-reel-scraper: Instagram 릴스 데이터 수집

분석 시 다음 항목을 포함하세요:
1. 콘텐츠 주제/카테고리 분석
2. 인기 콘텐츠 특징 (좋아요, 댓글, 조회수)
3. 해시태그 전략
4. 게시 빈도 및 패턴
5. 개선 제안

항상 한국어로 답변하세요.
"""

print("✅ Tools 및 LLM 설정 완료!")

ModuleNotFoundError: No module named 'langchain_apify'

In [ ]:
# 노드 함수 정의
def call_model(state: AgentState):
    """LLM 호출"""
    messages = state["messages"]
    
    # System prompt 추가
    if not any(isinstance(m, SystemMessage) for m in messages):
        messages = [SystemMessage(content=SYSTEM_PROMPT)] + messages
    
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}


def process_results(state: AgentState):
    """스크래핑 결과 후처리"""
    messages = state["messages"]
    
    # 마지막 Tool 결과에서 데이터 추출
    for msg in reversed(messages):
        if hasattr(msg, 'content') and isinstance(msg.content, str):
            if 'likesCount' in msg.content or 'commentsCount' in msg.content:
                # 데이터가 있으면 저장
                return {"instagram_data": msg.content}
    
    return {"instagram_data": {}}

print("✅ 노드 함수 정의 완료!")

In [ ]:
# StateGraph 구축
def build_instagram_agent():
    """Instagram 분석 Agent 그래프 생성"""
    
    builder = StateGraph(AgentState)
    
    # 노드 추가
    builder.add_node("agent", call_model)
    builder.add_node("tools", ToolNode(tools))
    builder.add_node("process", process_results)
    
    # 엣지 추가
    builder.add_edge(START, "agent")
    
    # 조건부 엣지: Tool 호출 여부에 따라 분기
    builder.add_conditional_edges(
        "agent",
        tools_condition,  # Tool 호출 필요하면 'tools', 아니면 END
        {
            "tools": "tools",
            END: END
        }
    )
    
    # Tool 실행 후 결과 처리
    builder.add_edge("tools", "process")
    builder.add_edge("process", "agent")
    
    return builder.compile()

# Agent 생성
instagram_agent = build_instagram_agent()
print("✅ Instagram Agent 그래프 생성 완료!")

In [ ]:
# 그래프 시각화 (선택사항)
try:
    from IPython.display import Image, display
    display(Image(instagram_agent.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"시각화 불가: {e}")
    print("\n📊 그래프 구조:")
    print("START → agent → (tools_condition)")
    print("  ├─ tools → process → agent")
    print("  └─ END")

In [ ]:
# 커스텀 Agent 실행
def analyze_influencer(username: str, limit: int = 5):
    """
    인플루언서 Instagram 릴스 분석
    
    Args:
        username: Instagram 사용자명
        limit: 분석할 릴스 개수
    """
    query = f"""
    {username} 인스타그램 계정의 최근 릴스 {limit}개를 분석해주세요.
    
    다음 정보를 포함해서 분석해주세요:
    - 콘텐츠 주제와 스타일
    - 인기 콘텐츠의 공통점
    - engagement 분석 (좋아요, 댓글, 조회수)
    - 해시태그 전략
    - 개선 제안
    """
    
    print(f"🔍 분석 대상: @{username}")
    print(f"📊 릴스 개수: {limit}개")
    print("=" * 60)
    
    result = instagram_agent.invoke({
        "messages": [HumanMessage(content=query)],
        "instagram_data": {},
        "analysis_result": ""
    })
    
    # 최종 응답 출력
    final_message = result["messages"][-1]
    print("\n" + "=" * 60)
    print("📋 분석 결과:")
    print("=" * 60)
    print(final_message.content)
    
    return result

print("✅ analyze_influencer 함수 정의 완료!")
print("\n사용법: analyze_influencer('natgeo', limit=5)")

In [ ]:
# 실행! (주석 해제하여 실행)
# result = analyze_influencer("natgeo", limit=5)

---

## 4️⃣ 실전 예제: 고급 인플루언서 분석 Agent

여러 기능을 통합한 완전한 분석 Agent를 구축합니다.

In [ ]:
from typing import Literal
from langchain_core.tools import tool
import json

# 커스텀 분석 도구 정의
@tool
def calculate_engagement_rate(likes: int, comments: int, views: int) -> str:
    """
    Engagement Rate를 계산합니다.
    
    Args:
        likes: 좋아요 수
        comments: 댓글 수  
        views: 조회수
    
    Returns:
        Engagement rate 분석 결과
    """
    if views == 0:
        return "조회수가 0이어서 계산할 수 없습니다."
    
    engagement_rate = ((likes + comments) / views) * 100
    
    # 벤치마크 기준
    if engagement_rate >= 6:
        grade = "🔥 매우 높음 (상위 5%)"
    elif engagement_rate >= 3:
        grade = "👍 높음 (상위 20%)"
    elif engagement_rate >= 1:
        grade = "📊 평균"
    else:
        grade = "📉 개선 필요"
    
    return f"""
Engagement Rate: {engagement_rate:.2f}%
등급: {grade}
- 좋아요: {likes:,}
- 댓글: {comments:,}
- 조회수: {views:,}
"""

@tool
def analyze_hashtags(hashtags: list[str]) -> str:
    """
    해시태그 전략을 분석합니다.
    
    Args:
        hashtags: 해시태그 리스트
    
    Returns:
        해시태그 분석 결과
    """
    if not hashtags:
        return "해시태그가 없습니다."
    
    total = len(hashtags)
    unique = len(set(hashtags))
    
    # 빈도 계산
    from collections import Counter
    counter = Counter(hashtags)
    top_5 = counter.most_common(5)
    
    result = f"""
📊 해시태그 분석:
- 총 사용: {total}개
- 고유 태그: {unique}개
- 평균 반복: {total/unique:.1f}회

🏆 Top 5 해시태그:
"""
    for tag, count in top_5:
        result += f"  #{tag}: {count}회\n"
    
    return result

print("✅ 커스텀 분석 도구 정의 완료!")

In [ ]:
# 고급 Agent 구성
from langchain_apify import ApifyActorsTool

# 모든 도구 통합
advanced_tools = [
    ApifyActorsTool("apify/instagram-reel-scraper"),  # 릴스 스크래핑
    ApifyActorsTool("apify/rag-web-browser"),         # 웹 검색
    calculate_engagement_rate,                         # 커스텀: engagement 계산
    analyze_hashtags,                                  # 커스텀: 해시태그 분석
]

# 고급 System Prompt
ADVANCED_SYSTEM_PROMPT = """
당신은 전문 Instagram 마케팅 분석가입니다.

## 사용 가능한 도구
1. **instagram-reel-scraper**: Instagram 릴스 데이터 수집
2. **rag-web-browser**: 웹 검색으로 추가 정보 수집  
3. **calculate_engagement_rate**: Engagement Rate 계산
4. **analyze_hashtags**: 해시태그 전략 분석

## 분석 프로세스
1. 먼저 instagram-reel-scraper로 릴스 데이터를 수집합니다
2. 수집된 데이터로 engagement_rate를 계산합니다
3. 해시태그가 있으면 analyze_hashtags로 분석합니다
4. 필요시 웹 검색으로 경쟁사 정보를 수집합니다
5. 종합적인 인사이트와 개선 제안을 제공합니다

## 출력 형식
항상 구조화된 형식으로 답변하세요:
- 📊 데이터 요약
- 📈 성과 분석
- 🎯 콘텐츠 전략
- 💡 개선 제안

한국어로 답변하세요.
"""

# 고급 Agent 생성
advanced_llm = ChatOpenAI(model="gpt-4o", temperature=0)  # 더 강력한 모델
advanced_agent = create_react_agent(
    advanced_llm.bind(system=ADVANCED_SYSTEM_PROMPT), 
    advanced_tools
)

print("✅ 고급 분석 Agent 생성 완료!")
print(f"🛠️ 도구 수: {len(advanced_tools)}개")

In [ ]:
async def full_analysis(username: str):
    """
    인플루언서 종합 분석 실행
    """
    query = f"""
    @{username} 인스타그램 계정을 종합 분석해주세요.
    
    수행할 작업:
    1. 최근 릴스 10개 데이터 수집
    2. 각 릴스의 Engagement Rate 계산
    3. 사용된 해시태그 분석
    4. 콘텐츠 트렌드 및 개선점 제안
    """
    
    print(f"🔍 종합 분석 시작: @{username}")
    print("=" * 60)
    
    async for event in advanced_agent.astream_events(
        {"messages": [HumanMessage(content=query)]},
        version="v2"
    ):
        # 도구 호출 이벤트 출력
        if event["event"] == "on_tool_start":
            print(f"\n🔧 도구 호출: {event['name']}")
        elif event["event"] == "on_tool_end":
            print(f"   ✅ 완료")
        elif event["event"] == "on_chat_model_stream":
            # 스트리밍 출력
            chunk = event["data"]["chunk"]
            if hasattr(chunk, 'content') and chunk.content:
                print(chunk.content, end="", flush=True)
    
    print("\n" + "=" * 60)
    print("✅ 분석 완료!")

print("✅ full_analysis 함수 정의 완료!")
print("\n사용법: await full_analysis('natgeo')")

In [ ]:
# 실행! (주석 해제하여 실행)
# await full_analysis("natgeo")

---

## 5️⃣ 프로덕션 배포 팁

### 5.1 환경 변수 관리

In [ ]:
# .env 파일 예시
env_example = """
# .env
APIFY_API_TOKEN=apify_api_xxxxxxxxxxxxx
OPENAI_API_KEY=sk-xxxxxxxxxxxxx
"""

# dotenv로 로드
load_env_code = """
from dotenv import load_dotenv
load_dotenv()

# 환경 변수 검증
import os
assert os.environ.get("APIFY_API_TOKEN"), "APIFY_API_TOKEN not set"
assert os.environ.get("OPENAI_API_KEY"), "OPENAI_API_KEY not set"
"""

print("📋 환경 변수 설정 예시:")
print(env_example)
print("\n📋 로드 코드:")
print(load_env_code)

### 5.2 에러 처리 및 재시도

In [ ]:
from tenacity import retry, stop_after_attempt, wait_exponential

@retry(
    stop=stop_after_attempt(3),
    wait=wait_exponential(multiplier=1, min=4, max=60)
)
async def robust_analysis(username: str):
    """
    재시도 로직이 포함된 분석 함수
    """
    try:
        result = await advanced_agent.ainvoke({
            "messages": [HumanMessage(content=f"{username} 릴스 분석해줘")]
        })
        return result
    except Exception as e:
        print(f"❌ 에러 발생: {e}")
        raise

print("✅ 재시도 로직 포함 함수 정의 완료!")

### 5.3 비용 최적화

In [ ]:
# 비용 최적화 팁
optimization_tips = """
## 💰 비용 최적화 전략

### Apify 비용 절감
1. resultsLimit 최소화: 필요한 만큼만 수집
2. 불필요한 옵션 비활성화:
   - includeTranscript: False
   - includeDownloadedVideo: False
   - includeSharesCount: False

### LLM 비용 절감
1. 간단한 작업: gpt-4o-mini 사용
2. 복잡한 분석: gpt-4o 사용
3. 캐싱 활용: 동일 쿼리 결과 저장

### 예시: 비용 효율적 설정
```python
# 기본 분석 (저비용)
basic_input = {
    "username": ["target"],
    "resultsLimit": 5,
    "includeSharesCount": False,
    "includeTranscript": False,
}

# 심층 분석 (고비용)
deep_input = {
    "username": ["target"],
    "resultsLimit": 30,
    "includeSharesCount": True,
    "includeTranscript": True,
}
```
"""

print(optimization_tips)

---

## 📚 요약 및 Reference

### 학습한 내용
| 방법 | 패키지 | 장점 | 사용 시점 |
|------|--------|------|----------|
| 방법 1 | `langchain-apify` | 간단, 빠른 설정 | 빠른 프로토타이핑 |
| 방법 2 | `langchain-mcp-adapters` | 표준 프로토콜, 확장성 | 프로덕션, 멀티 서버 |
| 방법 3 | Custom StateGraph | 완전한 제어 | 복잡한 워크플로우 |

### 🔗 참고 자료
- [Apify MCP 문서](https://docs.apify.com/platform/integrations/mcp)
- [LangGraph 문서](https://langchain-ai.github.io/langgraph/)
- [langchain-mcp-adapters GitHub](https://github.com/langchain-ai/langchain-mcp-adapters)
- [MCP 프로토콜 스펙](https://modelcontextprotocol.io/)

### 🎯 다음 단계
1. 다른 Apify Actor 연동 (TikTok, YouTube 등)
2. LangGraph Cloud 배포
3. 멀티 에이전트 시스템 구축
4. Streamlit/Gradio UI 추가

In [ ]:
print("🎉 튜토리얼 완료!")
print("\n이제 자연어로 인플루언서 분석을 요청하면 Agent가 알아서 처리합니다.")
print('\n예시: analyze_influencer("natgeo", limit=10)')